
# Classification Evaluation Metrics


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime

import random


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:
#%run "C040_MDL_NN w TensorFlow.ipynb"

In [ ]:

#preds_dataset_name = 'predictions_dataset_2022-10-18.csv'
#preds_dataset_name = 'predictions_dataset_2022-10-27.csv'
preds_dataset_name = 'predictions_dataset_2022-11-24.csv'

df_eval_raw = pd.read_csv(f'{fileloc}/{preds_dataset_name}')

print(df_eval_raw.shape)
print()
print(df_eval_raw.columns.values.tolist())


In [9]:
# Plotting the RaceGrade

In [ ]:

print(pd.DataFrame(df_eval_raw.isna().sum()).transpose())


In [ ]:

df_eval_raw.describe()


In [ ]:

df_eval_raw.head()


In [ ]:

df_evaluation = df_eval_raw.copy()


In [ ]:

# groupings
groupings = ['@id_RACE']

## Rank for Place Preds
#df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=False)
df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=True)

# Winner / Place Preds Columns
df_evaluation['Winner_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 1 else 0, axis = 1)
df_evaluation['Top3_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)

print(datetime.now())


In [ ]:

df_poscount = df_evaluation.groupby(groupings).agg({'position_preds':'nunique'}).reset_index().rename(columns= {'position_preds':'poscount'})


In [ ]:

df_eval_v2 = df_evaluation.merge(df_poscount, on = groupings, how = 'left')

df_eval_v2['preds_tie'] = df_eval_v2.apply(lambda x : 0 if x.x_num_dogs == x.poscount \
                                           else 1 if x.x_num_dogs == x.poscount + 1 \
                                           else 2 if x.x_num_dogs == x.poscount + 2 \
                                           else 3 if x.x_num_dogs == x.poscount + 3 \
                                           else 4 if x.x_num_dogs == x.poscount + 4 \
                                           else 5 if x.x_num_dogs == x.poscount + 5 \
                                           else 6 if x.x_num_dogs == x.poscount + 6 \
                                           else 7 if x.x_num_dogs == x.poscount + 7 \
                                           else 8 if x.x_num_dogs == x.poscount + 8 else 99, axis = 1)

print()

print(df_eval_v2.shape)
print(df_eval_v2.columns.values.tolist())


In [ ]:

print()
print(df_eval_v2.groupby(['preds_tie']).agg({'@id_RACE':'nunique'}).reset_index())


In [ ]:

#df_evaluation[df_evaluation['@id_RACE'].isin([626271182])].head()
df_eval_v2[df_eval_v2['ttflag'].isin(['Test'])].head()


In [ ]:

df_eval_v2.describe()


In [ ]:

df_eval_v2.sort_values(by = ['@id_RACE','position']).head()


In [ ]:

pd.crosstab(df_eval_v2.position, df_eval_v2.position_preds)


In [ ]:

df_eval_v2.groupby( [ 'ttflag', 'Top3_actual', 'Top3_preds']).agg({'@id_DOG':'count'}).reset_index()


In [ ]:

pd.crosstab(df_eval_v2.Top3_actual, df_eval_v2.Top3_preds, normalize='all')


In [ ]:

df_eval_v2.tail(10)


In [ ]:

print(datetime.now())


In [ ]:

# Random


In [ ]:


# Bet on all Preds = s1
df_eval_v2['rand_1'] = df_eval_v2.apply( lambda x : 1 if x.position_random == 1 else 0, axis = 1 )
df_eval_v2['h_rand_1'] = df_eval_v2.apply( lambda x : 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_eval_v2['p_rand_1'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )

print()
print(datetime.now())

# Bet on all Place = s2
#df_eval_v2['fs_2'] = df_eval_v2.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_eval_v2["rand_all_plc"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_random) \
                                                                else 1 if (x.x_num_dogs >= 8) & (x.position_random <= 3) \
                                                                    else 1 if (x.x_num_dogs >= 5) & (x.position_random <= 2) \
                                                                        else 0 if (x.x_num_dogs < 5) \
                                                                            else 0, axis = 1)
df_eval_v2['h_rand_all_plc'] = df_eval_v2.apply( lambda x : 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_eval_v2['p_rand_all_plc'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())


# Laying Last 
df_eval_v2["rand_lay_last"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_random) \
                                                                else 1 if (x.x_num_dogs == x.position_random) \
                                                                   else 0, axis = 1)

df_eval_v2['h_rand_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.rand_lay_last == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_eval_v2['p_rand_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.rand_lay_last == 1 ) & ( x.position > 1 ) \
                                             else -x.ft_sp*5 if ( x.rand_lay_last == 1 ) & ( x.position == 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())


In [ ]:

# Benchmark


In [ ]:

# Bet on all Preds = s1
df_eval_v2['ft_1'] = df_eval_v2.apply( lambda x : 1 if x.position_fasttrack == 1 else 0, axis = 1 )
df_eval_v2['h_ft_1'] = df_eval_v2.apply( lambda x : 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_eval_v2['p_ft_1'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )

print()
print(datetime.now())

# Bet on all Place = s2
df_eval_v2["ft_all_plc"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_fasttrack) \
                                                                else 1 if (x.x_num_dogs >= 8) & (x.position_fasttrack <= 3) \
                                                                    else 1 if (x.x_num_dogs >= 5) & (x.position_fasttrack <= 2) \
                                                                        else 0 if (x.x_num_dogs < 5) \
                                                                            else 0, axis = 1)
df_eval_v2['h_ft_all_plc'] = df_eval_v2.apply( lambda x : 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_eval_v2['p_ft_all_plc'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())

# Laying Last 
df_eval_v2["ft_lay_last"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_fasttrack) \
                                                                else 1 if (x.x_num_dogs == x.position_fasttrack) \
                                                                   else 0, axis = 1)

df_eval_v2['h_ft_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.ft_lay_last == 1 ) & ( x.position > 1) \
                                             else 0 , axis = 1 )

df_eval_v2['p_ft_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.ft_lay_last == 1 ) & ( x.position > 1 ) \
                                             else -x.ft_sp*5 if ( x.ft_lay_last == 1 ) & ( x.position == 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())
#3Min runtime


In [ ]:

# Strategies


In [ ]:

# Bet on all Preds = s1
df_eval_v2['s_1'] = df_eval_v2.apply( lambda x : 1 if x.position_preds == 1 else 0, axis = 1 )
df_eval_v2['h_s_1'] = df_eval_v2.apply( lambda x : 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_eval_v2['p_s_1'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


print()
print(datetime.now())

# Bet on all Place = s2
#df_eval_v2['fs_2'] = df_eval_v2.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_eval_v2["s_all_plc"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_preds) \
                                                                else 1 if (x.x_num_dogs >= 8) & (x.position_preds <= 3) \
                                                                    else 1 if (x.x_num_dogs >= 5) & (x.position_preds <= 2) \
                                                                        else 0 if (x.x_num_dogs < 5) \
                                                                            else 0, axis = 1)

df_eval_v2['h_s_all_plc'] = df_eval_v2.apply( lambda x : 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_eval_v2['p_s_all_plc'] = df_eval_v2.apply( lambda x : x.ft_sp - 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())

# Laying Last 
df_eval_v2["s_lay_last"] = df_eval_v2.apply(lambda x : np.nan if pd.isna(x.position_fasttrack) \
                                                                else 1 if (x.x_num_dogs == x.position_fasttrack) \
                                                                   else 0, axis = 1)

df_eval_v2['h_s_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.s_lay_last == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_eval_v2['p_s_lay_last'] = df_eval_v2.apply( lambda x : 1 if ( x.s_lay_last == 1 ) & ( x.position > 1 ) \
                                             else -x.ft_sp*5 if ( x.s_lay_last == 1 ) & ( x.position == 1 ) \
                                             else 0 , axis = 1 )

print()
print(datetime.now())
#3min runtime


In [ ]:

df_eval_v2.head()


In [ ]:

pd.set_option('display.max_rows', 50)


In [ ]:

# Correlations


In [ ]:

print('Position vs Random')
print(df_eval_v2['position'].corr( df_eval_v2['position_random'] ))
print()

print('Position vs Preds')
print(df_eval_v2['position'].corr( df_eval_v2['position_preds'] ))
print()

print('Position vs FastTrack')
print(df_eval_v2['position'].corr(df_eval_v2['position_fasttrack'] ))

print()
print()
print('Preds vs Fasttrack')
print(df_eval_v2['position_preds'].corr(df_eval_v2['position_fasttrack'] ))


In [ ]:

# Classic overestimation of Placement problems -- placing top 3 for 8, and also 3 for 7,6,5,4,3,2,1 racers (fix)
# Classic using WIN price for PLACE issue

df_eval = df_eval_v2.groupby(['preds_tie','ttflag'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum', 'ft_lay_last':'sum', 'h_ft_lay_last':'sum', 'p_ft_lay_last':'sum' ''\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum', 's_lay_last':'sum', 'h_s_lay_last':'sum', 'p_s_lay_last':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum', 'rand_lay_last':'sum', 'h_rand_lay_last':'sum', 'p_rand_lay_last':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()\
.transpose()


In [ ]:

df_eval


In [ ]:

#df_eval.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_totals.csv', index = True)
#df_eval.to_csv(f'{fileloc}/evaluation_2022-10-27_extended_totals.csv', index = True)
df_eval.to_csv(f'{fileloc}/evaluation_2022-11-24_extended_totals.csv', index = True)


In [ ]:

df_res = df_eval_v2.groupby(['preds_tie','ttflag','Track','RaceGrade_H1'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum'\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()


In [ ]:

df_res


In [ ]:

#df_res.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_bytrack.csv', index = True)
#df_res.to_csv(f'{fileloc}/evaluation_2022-10-27_extended_bytrack.csv', index = True)
df_res.to_csv(f'{fileloc}/evaluation_2022-11-24_extended_bytrack.csv', index = True)


In [115]:

print("THE END")


THE END
